In [12]:
!pip install langchain_neo4j

   ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
    --------------------------------------- 0.3/13.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.0 MB 1.3 MB/s eta 0:00:10
   -- ------------------------------------- 0.8/13.0 MB 987.4 kB/s eta 0:00:13
   -- ------------------------------------- 0.8/13.0 MB 987.4 kB/s eta 0:00:13
   --- ------------------------------------ 1.0/13.0 MB 799.2 kB/s eta 0:00:15
   ---- ----------------------------------- 1.3/13.0 MB 849.7 kB/s eta 0:00:14
   ---- ----------------------------------- 1.6/13.0 MB 902.1 kB/s eta 0:00:13
   ----- ---------------------------------- 1.8/13.0 MB 958.5 kB/s eta 0:00:12
   ----- ---------------------------------- 1.8/13.0 MB 958.5 kB/s eta 0:00:12
   ------- -------------------------------- 2.4/13.0 MB 994.4 kB/s eta 0:00:11
   --------

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.1 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.1 which is incompatible.


In [9]:
from neo4j import GraphDatabase
import os 
from langchain.graphs import Neo4jGraph

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"

uri = os.getenv('NEO4J_URI')
user_name = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

graph = Neo4jGraph(
    url="neo4j+s://9c98656c.databases.neo4j.io", 
    username="neo4j", 
    password="L6degoRAS1IO9E3ZMzeEujWfW0gggYkY1yXAEdJKLYQ"
)

In [ ]:
import requests

url = "https://gist.githubusercontent.com/tomasonjo/08dc8ba0e19d592c4c3cde40dd6abcc3/raw/da8882249af3e819a80debf3160ebbb3513ee962/microservices.json"
import_query = requests.get(url).json()['query']
graph.query(import_query)


[]

In [10]:
url="neo4j+s://9c98656c.databases.neo4j.io"
username="neo4j"
password="L6degoRAS1IO9E3ZMzeEujWfW0gggYkY1yXAEdJKLYQ"

In [7]:
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings import HuggingFaceEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
    url=url,
    username=username,
    password=password,
    index_name='tasks',
    node_label="Task",
    text_node_properties=["name", "description", "status"],  # Make sure this is a list
    embedding_node_property="embedding",
)


AttributeError: 'tuple' object has no attribute 'decode'

In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document
from langchain_neo4j import Neo4jVector
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

In [12]:
loader = TextLoader("text.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


In [52]:
vector_index = Neo4jVector.from_documents(
    documents=docs,
    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
    url=url,
    username=username,
    password=password,
    index_name='valorant_game_index',
    node_label='Game',
    embedding_node_property='embedding'
)

In [54]:
response = vector_index.similarity_search(
    "What is valorant",
)
#print(response[0].page_content)
# name: BugFix
# description: Add a new feature to RecommendationService to provide ...
# status: In Progress
print(response[0].page_content)

Valorant is a free-to-play 5v5 tactical first-person shooter (FPS) developed and published by Riot Games. Released officially on June 2, 2020, it's available on Windows, with a console version (PS5, Xbox Series X/S) launched in 2024 (though PC and console do not share cross-play to ensure fairness).

The game combines precise gunplay (similar to CS:GO) with unique character abilities (inspired by Overwatch), creating a fast-paced yet highly strategic competitive experience.

Core Mode (Unrated/Competitive): Attackers try to plant the Spike (a bomb), and defenders attempt to stop or defuse it. First to 13 rounds wins.

Economy System: Each player earns money per round to buy weapons and abilities.

Abilities + Gunplay: Each Agent has a set of abilities (including an Ultimate), but success still heavily relies on accurate shooting.

As of 2025, Valorant features 25+ Agents across 4 roles:


In [48]:
from langchain.chains import RetrievalQA
from langchain.chat_models import init_chat_model

vector_qa = RetrievalQA.from_chain_type(
    llm=init_chat_model("groq:llama-3.3-70b-versatile"),
    chain_type="stuff",
    retriever=vector_index.as_retriever()
)
vector_qa.run(
    "What is Valorant"
)

'Valorant is a free-to-play, tactical first-person shooter game developed by Riot Games, the same company that created League of Legends. It was released on June 2, 2020, for Microsoft Windows.\n\nIn Valorant, players are divided into two teams of five players each: Attackers and Defenders. The Attackers\' goal is to plant a bomb, called a "Spike," at one of the designated sites on the map, while the Defenders must prevent them from doing so or defuse the Spike if it\'s already been planted.\n\nThe game features a variety of playable characters, known as "Agents," each with unique abilities and playstyles. These Agents can be used to gain a strategic advantage in the game, such as healing teammates, blocking enemy movements, or revealing enemy positions.\n\nValorant is known for its competitive gameplay, precise gunplay, and strategic team coordination, making it a popular game among fans of first-person shooters and esports.'